# AWS STAC RTC

Start exploring this dataset with Xarray

In [ ]:
!pip -q install odc.stac

In [ ]:
import yaml
import odc.stac
import pystac
import hvplot.xarray

In [ ]:
# Paste /proxy/localhost:8787 for cluster diagnostics
from dask.distributed import Client
client = Client()
client

In [ ]:
# GDAL environment variables for better performance
import os
os.environ['AWS_REGION']='us-west-2'
os.environ['GDAL_DISABLE_READDIR_ON_OPEN']='EMPTY_DIR' 
os.environ['AWS_NO_SIGN_REQUEST']='YES' 

In [ ]:
# temporary fix https://github.com/opendatacube/odc-stac/issues/9#issuecomment-952363783
cfg = """---
"*":
  warnings: ignore # Disable warnings about duplicate common names

sentinel1-rtc-aws:
  assets:
    '*':
      data_type: float32
      nodata: 0
    'incidence':
      data_type: uint16
      nodata: 0
#      scale: 0.01 #need to do this manually
      
"""
cfg = yaml.load(cfg, Loader=yaml.CSafeLoader)

In [ ]:
cat = pystac.read_file('catalog.json')
items = list(cat.get_all_items())
ds = odc.stac.load(items,
                   #bands=["gamma0_vv", "gamma0_vh"],
                   stac_cfg=cfg,
                   chunks=dict(x=512, y=512, time=1),
                  )
print('Total dataset size =', ds.nbytes/1e9)
ds

In [ ]:
# Subset around Grand Mesa
xmin,xmax,ymin,ymax = [739186, 742748, 4.325443e+06, 4.327356e+06]

daT = ds['gamma0_vv'].sel(x=slice(xmin, xmax),
                          y=slice(ymax, ymin))

daT

In [ ]:
%%time
# Our area of interest is now small, and will easily fit in-memory
daT = daT.compute()

all_points = daT.where(daT!=0).hvplot.scatter('time', groupby=[], dynspread=True, datashade=True) 
mean_trend = daT.where(daT!=0, drop=True).mean(dim=['x','y']).hvplot.line(title='North Grand Mesa', color='red')
(all_points * mean_trend)

## Spatial visualizations

In [ ]:
i = 0
title=ds.time.values[0].astype('str')

ds['incidence'].isel(time=i).hvplot.image(rasterize=True,
                                          data_aspect=1,
                                          title=title,
                                          cmap='viridis',
                                          clabel='incidence (degrees)')

In [ ]:
ds['gamma0_vv'].isel(time=i).hvplot.image(rasterize=True,
                                          data_aspect=1,
                                          title=title,
                                          clim=(0,0.5),
                                          cmap='gray',
                                          clabel='gamma0_vv (watts)')

In [ ]:
ds['gamma0_vh'].isel(time=i).hvplot.image(rasterize=True,
                                          data_aspect=1,
                                          title=title,
                                          clim=(0,0.2), # not equal to vv scale
                                          cmap='gray',
                                          clabel='gamma0_vh (watts)')